# **Project Overview: Adapter-Based Vision-Language Fine-Tuning with Gemma-3N**

This project explores domain-specific fine-tuning of a Vision-Language Large Language Model (VL-LLM) for medical imaging tasks using LoRA adapters. I specifically target two different visual domains:


1.   Skin disease images
2.   Radiology images


The base model used for this work is:
unsloth/gemma-3n-E2B-it — a Vision-Language capable LLM that supports both image and text inputs, optimized for multi-modal tasks and efficient fine-tuning using LoRA adapters.





#**Objectives**



1.   Evaluate baseline performance of the LLM backbone without any fine-tuning.
2.  Fine-tune the model separately on:
    
        Skin dataset

        Radiology dataset

3.   Evaluate the performance of both fine-tuned models independently.
6.   Implement an embedding-based adapter selection method using CLIP + ViT to dynamically choose the correct domain adapter based on image similarity.
7.   Evaluate the accuracy of this adapter selection mechanism.
















**Important Note**

Since Colab GPU has limited RAM, it cannot hold multiple large models in memory at the same time.

So after we finish training or loading one model, we may(most probably not) need to **reset the GPU** to free up memory.

This means:
Every time we restart the runtime or clear the GPU memory, we must re-run:
All required imports (libraries like torch, unsloth, etc.)

 Dataset loading and preprocessing (like loading skin_test, radio_test, etc.)

Model definition code (e.g., loading base models, adapters, processors)

This ensures the notebook has everything in memory again and avoids errors like:

    CUDA out of memory
    Model or tokenizer not defined

So remember to **always re-run the earlier cells** after a reset to make sure everything works smoothly!

I have also added the comment "Run Everytime" on those cells


In [ ]:
# ⚠️ I use this to forcefully reset the Colab runtime and fully clear GPU memory.
# Since I fine-tune two adapters (skin & radiology) on the same base model, training both in one session cannot be possible.
# After saving one adapter, this line restarts the session so I can train the next one cleanly.
import os
os.kill(os.getpid(), 9)  # Reset Colab runtime to free all memory


UnslothVisionDataCollator depends on ConstantLengthDataset, which exists in older TRL version, TRL has released new version on 29 july-2025 midnight, so run this package if you facing error while runnig the library cell, Run next 2 cells if you are facing dependence issues, which happened after latest update

In [ ]:
#run only if getting dependensy issues
# Uninstall any existing TRL version
!pip uninstall -y trl

# Force install the desired version
!pip install trl==0.19.1 --force-reinstall



Found existing installation: trl 0.20.0
Uninstalling trl-0.20.0:
  Successfully uninstalled trl-0.20.0
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached transformers-4.54.0-py3-none-any.whl.metadata (41 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.2 MB/s eta 0:00:00
  Using cached huggingface_hub-0.34.2-py3-none-any.whl.metadata (14 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 28.8 MB/s eta 0:00:00
Using cached datasets-4.0.0-py3-none-any.whl (494 kB)
Using cached transformers-4.54.0-py3-none-any.whl (11.2 MB)
   ━━━━━━━━━━━━━

In [ ]:
# Clean install of torch and torchvision with CUDA support
#run only if getting dependensy issues
!pip uninstall -y torch torchvision
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118


Found existing installation: torch 2.7.1
Uninstalling torch-2.7.1:
  Successfully uninstalled torch-2.7.1
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 821.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#it shold be less then 0.20
#run only if getting dependensy issues
import trl
print(trl.__version__)


0.19.1


In [ ]:
#Run Everytime
# Install the Hugging Face `datasets` library (or upgrade it if already installed).
# I use it to load the skin and radiology datasets from the Hugging Face Hub.
!pip install -U datasets
# Install the Hugging Face `transformers` library.
!pip install transformers

Unsloth makes fine-tuning large language models fast and memory-efficient by supporting 4-bit quantization, LoRA adapters, adapter merging, gradient checkpointing, and seamless integration with Hugging Face — perfect for limited environments like Google Colab.


In [ ]:
#Run Everytime
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.19.1 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
#Run Everytime
%%capture
# Install latest transformers for Gemma 3N
!pip install --no-deps transformers==4.53.1 # Only for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N

In [ ]:
#Run Everytime
#libraries

from datasets import load_dataset
import random
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig
from transformers import TextStreamer
import torch
from unsloth import FastVisionModel # FastLanguageModel for LLMs
from peft import PeftModel
from transformers import AutoImageProcessor
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image
import torch.nn.functional as F


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
import importlib.metadata as metadata

packages = [
    "unsloth",
    "transformers",
    "timm",
    "trl",
    "peft",
    "torch",
    "torchvision",
    "datasets",
    "sentencepiece",
    "protobuf",
    "huggingface_hub",
    "hf_transfer",
    "bitsandbytes",
    "accelerate",
    "xformers",
    "triton",
    "cut_cross_entropy",
]

for package in packages:
    try:
        version = metadata.version(package)
        print(f"{package}: {version}")
    except metadata.PackageNotFoundError:
        print(f"{package}: NOT INSTALLED")


unsloth: 2025.7.11
transformers: 4.53.1
timm: 1.0.19
trl: 0.19.1
peft: 0.16.0
torch: 2.6.0+cu124
torchvision: 0.21.0+cu124
datasets: 3.6.0
sentencepiece: 0.2.0
protobuf: 5.29.5
huggingface_hub: 0.33.5
hf_transfer: 0.1.9
bitsandbytes: 0.46.1
accelerate: 1.9.0
xformers: 0.0.29.post3
triton: 3.2.0
cut_cross_entropy: 25.1.1




### Model Details: `unsloth/gemma-3n-E2B-it`, Base Model

This function initializes a vision-language model using the `unsloth/gemma-3n-E2B-it` checkpoint, a fine-tuned variant of Google's Gemma model adapted for efficient multimodal tasks. The model is loaded using the FastVisionModel API from the UnsLoTH framework, optimized for low-memory and long-context scenarios.

* **Model Checkpoint**: `unsloth/gemma-3n-E2B-it`
  This is a vision-language instruction-tuned model, designed for efficient inference and training on multimodal tasks such as image captioning, VQA, and instruction following with visual context.

* **4-bit Quantization**:
  The model is loaded in 4-bit precision (`load_in_4bit=True`), reducing memory consumption significantly while maintaining acceptable performance. This is ideal for environments with limited GPU memory.

* **Gradient Checkpointing**:
  Using `"unsloth"` as the `use_gradient_checkpointing` setting enables optimized gradient checkpointing routines provided by UnsLoTH, allowing the model to process longer sequences with reduced memory usage.

* **Returns**:
  The function returns two components:

  * `model`: The quantized and initialized vision-language model.
  * `processor`: The corresponding processor/tokenizer responsible for pre-processing inputs and post-processing outputs.

This setup is intended to be executed every time the runtime is started, ensuring the model and processor are correctly instantiated in memory for subsequent training or inference workflows.


In [ ]:
#Run Everytime
def BaseModel():
  model, processor = FastVisionModel.from_pretrained(
    "unsloth/gemma-3n-E2B-it",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
    )
  return model,processor


### What `convert_to_conversation(sample)` Does

This function takes a single example that includes a question, an image, and an answer, and turns it into a conversation format that vision-language models can understand.

It builds a simple back-and-forth between a "user" and an "assistant":

* The user provides a **question** along with an **image**.
* The assistant replies with a **text answer**.

The function wraps this exchange into a list of messages under the key `"messages"`. This format is useful for models trained to understand multi-turn conversations, especially ones that take both text and images as input.

Run this every time you prepare a new sample for training or inference. It helps convert raw data into a structured dialogue format that matches what the model expects.



In [ ]:
#Run Everytime
def convert_to_conversation(sample):
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": sample["question"]},
                {"type": "image", "image": sample["image"]},
            ],
        },
        {"role": "assistant", "content": [{"type": "text", "text": sample["answer"]}]},
    ]
    return {"messages": conversation}
pass



### Dataset Setup

This section loads two datasets for training and testing:

* **Skin Dataset**: [`sercetexam9/skincap`](https://huggingface.co/datasets/sercetexam9/skincap)
  A dataset containing skin disease images, used for visual diagnosis tasks.

  * First 1500 samples are used for training.
  * Samples from index 1500 to 1549 are set aside for testing (50 samples).

* **Radiology Dataset**: [`unsloth/Radiology_mini`](https://huggingface.co/datasets/unsloth/Radiology_mini)
  Contains radiology images and associated data for medical imaging tasks.

  * First 1500 samples used for training.
  * 50 samples from index 1500 to 1549 used for testing.

This setup ensures a consistent split for both datasets, using the same start and end indices. These datasets do not include questions, questions will be added later in the pipeline.




In [ ]:
#Run Everytime
#dataset without questions
start, end = 1700, 1750  # end is exclusive
# SKin Dataset

dataset_skin = load_dataset("sercetexam9/skincap", split="train")
skin_test = dataset_skin.select(range(start, end))
dataset_skin = dataset_skin.select(range(1500))

#Radiology dataset
dataset_radio = load_dataset("unsloth/Radiology_mini", split="train")
radio_test = dataset_radio.select(range(start, end))
dataset_radio = dataset_radio.select(range(1500))



**Add synthetic Question to Dataset**

This part adds synthetic questions and formats the skin and radiology datasets for training and testing.
Each image gets a predefined expert-level question and keeps its original answer.
Unneeded columns like `text`, `caption`, or `image_id` are removed during mapping.
Testing sets only keep the image and answer for evaluation purposes.


In [ ]:
#Run Everytime
#skin part training
synthetic_questions_skin = [
    "You are an expert dermatologist. Describe accurately what you see in this image."
]
# Map over dataset to add Q&A
def add_skin_fields(example):
    return {
        "image": example["image"],
        "question": synthetic_questions_skin,
        "answer": example["text"]  # Or whatever text field your dataset uses
    }

dataset_skin = dataset_skin.map(add_skin_fields,remove_columns=["text"])



#radiology part training
synthetic_questions_radio = [
   "You are an expert radiologist. Describe accurately what you see in this image."

]

# Map over dataset to add Q&A
def add_radio_fields(example):
    return {
        "image": example["image"],
        "question": synthetic_questions_radio,
        "answer": example["caption"]
    }

dataset_radio = dataset_radio.map(add_radio_fields,remove_columns=["image_id","cui"])






Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:

#testing data
def add_radio_test_fields(example):
    return {
        "image": example["image"],
        "answer": example["caption"]
    }

radio_test = radio_test.map(add_radio_test_fields,remove_columns=["image_id","cui"])


def add_skin_test_fields(example):
    return {
        "image": example["image"],
        "answer": example["text"]
    }

skin_test = skin_test.map(add_skin_test_fields,remove_columns=["text"])


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
#checking the dataset is loaded
print(dataset_skin[1])
print(dataset_radio[1])

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=921x678 at 0x7A93F2158450>, 'question': ['You are an expert dermatologist. Describe accurately what you see in this image.'], 'answer': "This description suggests the presence of white patches on the skin. It is recommended to perform Wood's lamp examination, dermatoscopy, or pathological examination to confirm whether it is vitiligo, anemia spots, or other diseases.. Skin tone: 56. Malignant: 1. Features: Papule, White(Hypopigmentation)"}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=750x752 at 0x7A93ED73D090>, 'caption': 'ERCP showing distal CBD compression. ERCP - endoscopic retrograde cholangiopancreatography; CBD - common bile duct', 'question': ['You are an expert radiologist. Describe accurately what you see in this image.'], 'answer': 'ERCP showing distal CBD compression. ERCP - endoscopic retrograde cholangiopancreatography; CBD - common bile duct'}


**Skin Model Part**

First, I will load the base model, then add LoRA for fine-tuning. After that, I’ll check the base model’s responses before training. Next, I’ll train the model and evaluate its output again to see the improvements. Finally, I’ll save the model and reset the Colab environment to start training the radiology model.

In [ ]:
#skin model
model_skin_temp, processor_skin=BaseModel()

==((====))==  Unsloth 2025.7.8: Fast Gemma3N patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/469M [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]



### 🔧 PEFT (LoRA) Configuration Explanation

This block sets up parameter-efficient fine-tuning (PEFT) on the base vision-language model using LoRA. Here's what each option means:

* **`model_skin_temp`**
  The base model on which applying LoRA to, Base model.

* **`finetune_vision_layers=True`**
  Enables fine-tuning for the vision encoder, so the model can learn to interpret images more accurately for the task.

* **`finetune_language_layers=True`**
  Allows the text (language) layers to be fine-tuned. Important for adapting the model’s responses to our medical domain.

* **`finetune_attention_modules=True`**
  Applies LoRA to the attention layers (like query/key/value). These layers are essential for understanding context in both vision and text.

* **`finetune_mlp_modules=True`**
  Applies LoRA to the MLP (feed-forward) layers inside each transformer block.

* **`r=16`**
  The LoRA rank. This controls how many parameters are added for adaptation. Higher = more capacity.

* **`lora_alpha=16`**
  A scaling factor for LoRA. Often set equal to `r` to balance the updates.

* **`lora_dropout=0`**
  No dropout is used during LoRA training. Keeps training deterministic and simple.

* **`bias="none"`**
  Bias terms are not trained.

* **`random_state=3407`**
  Sets a fixed seed for reproducibility. Ensures consistent results each time you run the cell.

* **`use_rslora=False`**
  Enables Rank-Stabilized LoRA, Skipping extra stability to speed-up fine tuning

* **`loftq_config=None`**
  No LoFTQ (quantization-aware fine-tuning) used. Skipping this keeps the setup straightforward.

* **`target_modules="all-linear"`**
  Applies LoRA to all linear (fully connected) layers. Useful for fully adapting the model while keeping things efficient.

* **`modules_to_save=["lm_head", "embed_tokens"]`**
  Only these modules will be saved after fine-tuning.

  * `lm_head`: Final layer used for generating text.
  * `embed_tokens`: Token embedding layer.
  
    This keeps the saved model lightweight and focused on what changed during training.



In [ ]:
#skin
model_skin = FastVisionModel.get_peft_model(
    model_skin_temp,                        # Base model to apply PEFT on
    finetune_vision_layers=True,      # Enable fine-tuning vision layers
    finetune_language_layers=True,    # Enable fine-tuning language layers
    finetune_attention_modules=True,  # Fine-tune attention modules
    finetune_mlp_modules=True,        # Fine-tune MLP modules

    r=16,                             # LoRA rank for lightweight adaptation
    lora_alpha=16,                    # LoRA learning rate parameter, better equal to r
    lora_dropout=0,                  # No dropout during LoRA fine-tuning
    bias="none",                    # No bias adjustment during training
    random_state=3407,              # Seed for reproducibility
    use_rslora=False,               # Disable Rank-Stabilized LoRA, Keeps training simpler and faster when stable training isn't critical.
    loftq_config=None,              # No LoFTQ configuration used, to Avoid extra complexity
    target_modules="all-linear",   # Target all linear modules, fine tune all linear layers
    modules_to_save=[              # Modules to save after fine-tuning
        "lm_head",
        "embed_tokens",
    ],
)


Unsloth: Making `model.base_model.model.model.language_model` require gradients


This line converts each skin dataset sample into the conversation format required by the model.


In [ ]:
#calling conversion function
converted_dataset_skin = [convert_to_conversation(sample) for sample in dataset_skin]

### Skin Model Inference Example before Training

This code prepares an image and instruction, formats them into a chat input, and runs the skin model to generate a detailed dermatology description.
It handles image preprocessing, applies the processor, and streams the model’s generated text output in real-time.


In [ ]:
#skin
FastVisionModel.for_inference(model_skin)  # Enable for inference!

image = dataset_skin[20]["image"]
instruction = "You are an expert dermatologist. Describe accurately what you see in this image."

messages = [
    {
        "role": "user",
        "content": [{"type": "image"}, {"type": "text", "text": instruction}],
    }
]
input_text = processor_skin.apply_chat_template(messages, add_generation_prompt=True)

# Convert grayscale image to RGB
if image.mode == 'L':
    image = image.convert('RGB')


inputs = processor_skin(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")



text_streamer = TextStreamer(processor_skin, skip_prompt=True)
result = model_skin.generate(**inputs, streamer = text_streamer, max_new_tokens = 256,
                        use_cache=True, temperature = 1.0, top_p = 0.95, top_k = 64)

Certainly! Here's a description of what I see in the image, presented from a dermatologist's perspective:

**Clinical Presentation:**

The image shows a localized skin lesion on the upper back. The lesion appears as a somewhat raised, erythematous (reddened) patch with a slightly raised border. There's a central area of more intense redness and some small, scattered, bluish-purple spots within the lesion. The skin surrounding the lesion is generally normal in appearance.

**Possible Differential Diagnoses:**

Based on this presentation, several possibilities come to mind, though a definitive diagnosis requires further clinical and potentially laboratory evaluation:

* **Erythema nodosum:** This is a common inflammatory condition characterized by painful, red nodules beneath the skin. While the appearance isn't perfectly consistent with the typical nodular form, it's a possibility.
* **Follicular psoriasis:** This is a form of psoriasis that affects the hair follicles, often presenting 

### Skin Model Training Setup

This code prepares and configures the skin model for fine-tuning using the SFT framework.

* **Switch to Training Mode**
  `FastVisionModel.for_training(model_skin)` activates the model’s training mode, enabling gradient updates and parameter tuning.

* **Trainer Initialization**
  The `SFTTrainer` is initialized with:

  * `model_skin`: The PEFT-adapted skin model to be fine-tuned.
  * `converted_dataset_skin`: The training dataset, already converted into the conversation format for the model.
  * `processor_skin.tokenizer`: Tokenizer used to process text data during training.
  * `UnslothVisionDataCollator`: Handles batching and preprocessing of image-text pairs, resizing images to 512 pixels.

* **Training Configuration (`SFTConfig`)**
  Specifies detailed training parameters:

  * `per_device_train_batch_size=1`: Uses batch size of 1 per GPU/device.
  * `gradient_accumulation_steps=4`: Accumulates gradients over 4 steps to simulate a batch size of 4.
  * `gradient_checkpointing=False`: Disables gradient checkpointing to trade memory for speed.
  * `max_grad_norm=0.3`: Gradient clipping to stabilize training, based on QLoRA research.
  * `warmup_steps=5`: Gradually increases learning rate during the first 5 steps to improve convergence.
  * `num_train_epochs=2`: Trains for 2 full passes over the dataset.
  * `learning_rate=2e-4`: Sets the optimizer learning rate.
  * `logging_steps=1`: Logs training metrics every step for detailed monitoring.
  * `save_strategy="steps"`: Saves model checkpoints periodically during training.
  * `optim="adamw_torch_fused"`: Uses a fused version of the AdamW optimizer for efficiency.
  * `weight_decay=0.01`: Applies weight decay regularization to prevent overfitting.
  * `lr_scheduler_type="cosine"`: Uses cosine annealing learning rate scheduler to smoothly decay learning rate.
  * `seed=3407`: Fixes the random seed for reproducibility.
  * `output_dir="outputs"`: Directory where model checkpoints and logs are saved.
  * `report_to="none"`: Disables external reporting (like Weights & Biases).

* **Vision Fine-tuning Specifics**
  Additional settings to optimize vision-language training:

  * `remove_unused_columns=False`: Keeps all dataset columns during training.
  * `dataset_text_field=""`: No specific text field is selected since data is already formatted.
  * `dataset_kwargs={"skip_prepare_dataset": True}`: Skips redundant dataset preparation steps.
  * `max_seq_length=2048`: Allows long sequences for text input during training.

Overall, this setup configures an efficient and stable training loop for fine-tuning the skin model on dataset, balancing speed, efficiency etc


In [ ]:
#skin
FastVisionModel.for_training(model_skin) # Enable for training!


trainer_skin = SFTTrainer(
    model=model_skin,
    train_dataset=converted_dataset_skin,
    processing_class=processor_skin.tokenizer,
    data_collator=UnslothVisionDataCollator(model_skin, processor_skin, resize=512),
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        gradient_checkpointing = False,
        max_grad_norm = 0.3,              # max gradient norm based on QLoRA paper
        warmup_steps = 5,                 # Use when using max_steps
        #max_steps = 30,#45
        num_train_epochs = 2,           # Set this instead of max_steps for full training runs
        learning_rate = 2e-4,
        logging_steps = 1,
        save_strategy="steps",
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",             # For Weights and Biases

        # For vision finetuning
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_seq_length = 2048,
    )
)

###Training the Model

In [ ]:
#training the model
trainer_stats_skin = trainer_skin.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,500 | Num Epochs = 2 | Total steps = 750
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 21,135,360 of 5,460,573,632 (0.39% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,10.200400
2,9.755300
3,9.523000
4,9.933500
5,9.626700
6,9.181800
7,9.209600
8,9.639500
9,10.244900
10,9.460200


###Inference After Model Training

Checking the result after training on same image

In [ ]:
#skin
FastVisionModel.for_inference(model_skin)  # Enable for inference!

image = dataset_skin[20]["image"]
instruction = "You are an expert determologist. Describe accurately what you see in this image."

messages = [
    {
        "role": "user",
        "content": [{"type": "image"}, {"type": "text", "text": instruction}],
    }
]
input_text = processor_skin.apply_chat_template(messages, add_generation_prompt=True)

# Convert grayscale image to RGB
if image.mode == 'L':
    image = image.convert('RGB')


inputs = processor_skin(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(processor_skin, skip_prompt=True)
result = model_skin.generate(**inputs, streamer = text_streamer, max_new_tokens = 256,
                        use_cache=True, temperature = 1.0, top_p = 0.95, top_k = 64)

A 1-cm diameter, well-circumscribed, erythematous, scaly, and slightly raised lesion with a central clearing and a small black dot in the center.<end_of_turn>


###Actual Answer
This is the actual answer that was use to train the model


In [ ]:
dataset_skin[20]["answer"]

'The patient presents with oval-shaped patches on the trunk with fine scaling on the surface, suggesting a possible diagnosis of pityriasis rosea or skin tumor. Further investigation and medical history are needed for a definitive diagnosis. Pityriasis rosea is a common chronic inflammatory skin condition characterized by red patches with scales and itching. Skin tumors refer to growths or lumps on the skin, which can be benign like lipomas or malignant like skin cancer. Prompt medical evaluation is recommended for accurate diagnosis and appropriate treatment.. Skin tone: 56. Malignant: 1. Features: Plaque, Scale, Erythema'

###Model Saving
Saving the Model along processor

In [ ]:
model_skin.save_pretrained("skin_model")  # Local saving so cna download while testing



#Compressing
This command compresses the saved skin model folder into a ZIP file.
Because Colab GPU sessions have limited time, zipping and downloading the model lets you save it externally  for later use in testing or comparison.


In [ ]:
!zip -r /content/skin_model.zip /content/skin_model

  adding: content/skin_model/ (stored 0%)
  adding: content/skin_model/adapter_model.safetensors (deflated 8%)
  adding: content/skin_model/adapter_config.json (deflated 63%)
  adding: content/skin_model/README.md (deflated 65%)


###Resetting Colab Session
Resetting colab session for Radiology Dataset

In [ ]:
import os
os.kill(os.getpid(), 9) #to reset colab to clear the gpu memory

###Radiology Model part

`Skipping the Comments & Explanation on this section, as same as Skin Part`

Following the same path as for Skin Model

In [ ]:
#Radiology model
model_radio_temp, processor_radio=BaseModel()

==((====))==  Unsloth 2025.7.11: Fast Gemma3N patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

...

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

Adding Lora

In [ ]:
#radiology
# Applies Parameter-Efficient Fine-Tuning (PEFT) to the model using LoRA,
# allowing fine-tuning of selected vision and language components while keeping the model lightweight.
# Useful for adapting large models to specific tasks with minimal overhead.

model_radio = FastVisionModel.get_peft_model(
    model_radio_temp, #BaseModel
    finetune_vision_layers=True,
    finetune_language_layers=True,
    finetune_attention_modules=True,
    finetune_mlp_modules=True,

    r=16, # lora adaption, small value light weight
    lora_alpha=16, #learing rate, better same as r
    lora_dropout=0,
    bias="none",
    random_state=3407,
    use_rslora=False, # If True, uses Rank-Stabilized LoRA, an enhanced LoRA variant, better True if using low rank, like 4
    loftq_config=None,
    target_modules="all-linear", #just finetune linear layers
    modules_to_save=[
        "lm_head",
        "embed_tokens",
    ],
)


Unsloth: Making `model.base_model.model.model.language_model` require gradients


In [ ]:
#calling dataset converion function
converted_dataset_radio = [convert_to_conversation(sample) for sample in dataset_radio]

Inference Before Model training

In [ ]:
#radiology
FastVisionModel.for_inference(model_radio)  # Enable for inference!

image = dataset_radio[20]["image"]#image selected for inference
instruction = "You are an expert Radiologist. Describe accurately what you see in this image."

messages = [
    {
        "role": "user",
        "content": [{"type": "image"}, {"type": "text", "text": instruction}],
    }
]
#applying chat template\
input_text = processor_radio.apply_chat_template(messages, add_generation_prompt=True) # as processor is same and unchanged

# Convert grayscale image to RGB
if image.mode == 'L':
    image = image.convert('RGB')


inputs = processor_radio(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(processor_radio, skip_prompt=True)
#tokenizing input data, streamer to display,temp to contrrol randomness , top p and k for randomness
result = model_radio.generate(**inputs, streamer = text_streamer, max_new_tokens = 256,
                        use_cache=True, temperature = 1.0, top_p = 0.95, top_k = 64)

## Radiologist Report: Angiogram

**Patient:** (Not specified in the image)
**Date:** 28.12.2020
**Study:** Angiogram

**Findings:**

The image demonstrates a series of tortuous and irregular branching vessels in the posterior cranial fossa. 

Specifically, I observe:

* **Multiple vessels:** Several vessels are visible, exhibiting varying sizes and configurations.
* **Tortuous course:** The vessels display a highly irregular and winding course, suggesting potential abnormalities in their development or trajectory.
* **Irregular branching:** There are numerous branching patterns, some appearing abnormal or atypical.
* **Posterior cranial fossa location:** The vessels are situated within the posterior cranial fossa, indicating a potential issue with the brainstem or posterior circulation.

**Impression:**

The findings are suggestive of a vascular anomaly within the posterior cranial fossa. Further clinical correlation and potentially additional imaging studies may be warranted to deter

Model Training Setup

In [ ]:
#radio
FastVisionModel.for_training(model_radio) # Enable for training!
# will up
trainer_radio = SFTTrainer(
    model=model_radio,
    train_dataset=converted_dataset_radio,
    processing_class=processor_radio.tokenizer,
    data_collator=UnslothVisionDataCollator(model_radio, processor_radio, resize=512),
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4, #will accumulate gradient over 4 forward and bakward before updating
        gradient_checkpointing = False,

        # use reentrant checkpointing
        # gradient_checkpointing_kwargs = {"use_reentrant": False},
        max_grad_norm = 0.3,              # max gradient norm based on QLoRA paper
        warmup_steps = 5,                 # Use when using max_steps
        #max_steps = 62,#46
        # warmup_ratio = 0.03,
        num_train_epochs = 2,           # Set this instead of max_steps for full training runs
        learning_rate = 2e-4,
        logging_steps = 1,
        save_strategy="steps",
        optim = "adamw_torch_fused",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",             # For Weights and Biases

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_seq_length = 2048,
    )
)


In [ ]:
#training
trainer_stats_radio = trainer_radio.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,500 | Num Epochs = 2 | Total steps = 750
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 21,135,360 of 5,460,573,632 (0.39% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,10.913400
2,11.531600
3,11.262900
4,10.363400
5,11.434300
6,11.216000
7,9.797300
8,10.825900
9,11.658600
10,11.017400


Inference After model training

In [ ]:
#radio
FastVisionModel.for_inference(model_radio)  # Enable for inference!

image = dataset_radio[20]["image"]
instruction = "You are an expert radiologist. Describe accurately what you see in this image."

messages = [
    {
        "role": "user",
        "content": [{"type": "image"}, {"type": "text", "text": instruction}],
    }
]
input_text = processor_radio.apply_chat_template(messages, add_generation_prompt=True)

# Convert grayscale image to RGB
if image.mode == 'L':
    image = image.convert('RGB')


inputs = processor_radio(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")



text_streamer = TextStreamer(processor_radio, skip_prompt=True)
result = model_radio.generate(**inputs, streamer = text_streamer, max_new_tokens = 256,
                        use_cache=True, temperature = 1.0, top_p = 0.95, top_k = 64)


Selective angiogram of the right middle cerebral artery showing a large aneurysm in the right middle cerebral artery.<end_of_turn>


In [ ]:
print("Actual Answer")
dataset_radio[20]["answer"]

Actual Answer


'Post embolisation angiogram using PVA particles showed a marked reduction in vascularity'

Saving the model to load it later

In [ ]:
model_radio.save_pretrained("radio_model")  # Local saving



###Compressing

In [ ]:

!zip -r /content/radio_model.zip /content/radio_model


  adding: content/radio_model/ (stored 0%)
  adding: content/radio_model/adapter_model.safetensors (deflated 8%)
  adding: content/radio_model/adapter_config.json (deflated 63%)
  adding: content/radio_model/README.md (deflated 65%)




### Model Testing Strategy

To evaluate performance, I will follow these steps:

1. **Load the Models**
   First, I will load the fine-tuned models (Skin and Radiology) either from Google Drive or the local Colab environment, depending on availability.

2. **Compute Embeddings for Mode Selection**
   I will compute the average embeddings of the skin and radiology Training datasets. During testing, each input image will be compared to these averages to determine whether it’s more similar to the skin or radiology domain. Based on this similarity, the system will dynamically select the appropriate model (Skin or Radiology) for generating a response.

3. **Test Each Model**
   I will run inference on 30 test samples for each of the fine-tuned models (Skin and Radiology) to evaluate their performance individually.

4. **Compare with the Base Model**
   For benchmarking, I will also test the same 30 samples using the base (unfine-tuned) model to compare the improvements.

5. **Evaluation Using Sentence Embeddings**
   To compare model outputs objectively, I will use the `SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')` to compute embeddings of the generated answers and compare them using **cosine similarity**. This provides a quantitative way to assess how close each model's output is to the expected answer.

In summary, I will  test all three models — **Base**, **Skin**, and **Radiology** — using embedding-based similarity scoring for consistent evaluation.


###Resetting Session to do Testing

In [ ]:
import os
os.kill(os.getpid(), 9)  # Kills the Colab runtime — fully resets memory



In [ ]:
#using it, to upload the previously trained model
from google.colab import drive
import zipfile
import os


drive.mount('/content/drive')


zip_path = "/content/drive/MyDrive/skin_model.zip"


extract_dir = "/content/drive/MyDrive/skin_model"


os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Extracted to: {extract_dir}")


Mounted at /content/drive
Extracted to: /content/drive/MyDrive/skin_model


In [ ]:
zip_path = "/content/drive/MyDrive/radio_model.zip"


extract_dir = "/content/drive/MyDrive/radio_model"


os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Extracted to: {extract_dir}")


Extracted to: /content/drive/MyDrive/radio_model


###Dynamic Embedding-Based Model Selection Accuracy

In this code, I evaluate how accurately the system can **dynamically classify images as either skin or radiology** based on their visual embeddings.

* For each test image, I compute its **CLIP embedding** using `get_image_embedding()`, and then compare it to the **average embedding** of the skin and radiology datasets using **cosine similarity**.
* The model selects the domain (skin or radiology) based on which average embedding the test image is more similar to.
* I repeat this for **100 samples from each test set also** (skin and radiology), making a total of 200 classification attempts.

At the end, the script calculates the accuracy of these predictions.

In this run, the dynamic model selection achieved **100% accuracy**, meaning all test images were correctly classified into their respective domains using this embedding-based method.


In [ ]:
# Load CLIP for embedding extraction
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model.eval()


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

###Calculating Embedding of training set, 200 images per set

In [ ]:
import torch
def get_image_embedding(image: Image.Image):

    inputs = clip_processor(images=image, return_tensors="pt")

    with torch.no_grad():
        embedding = clip_model.get_image_features(**inputs)
    return F.normalize(embedding, p=2, dim=-1)  # Normalize for cosine similarity

skin_images = []
radio_images = []
#selecting images from both sets, radiology
for example in dataset_radio:
    img = example["image"]
    label = False

    if len(radio_images) < 200:
        radio_images.append(img)
    if len(radio_images) >= 200:
        break
#Skin dataset
for example in dataset_skin:
    img = example["image"]
    label = False

    if len(skin_images) < 200:
        skin_images.append(img)
    if  len(skin_images) >= 200:
        break


#calling embedding function and saving them
avg_skin = torch.mean(torch.stack([get_image_embedding(img) for img in skin_images]), dim=0)
avg_radio = torch.mean(torch.stack([get_image_embedding(img) for img in radio_images]), dim=0)

##Finding Accuracy of dynamic Selection

In [ ]:
correct = 0
total = 0



def get_image_embedding(image: Image.Image):

    inputs = clip_processor(images=image, return_tensors="pt")

    with torch.no_grad():
        embedding = clip_model.get_image_features(**inputs)
    return F.normalize(embedding, p=2, dim=-1)  # Normalize for cosine similarity

# Classify and evaluate skin images
for item in skin_test.select(range(50)):  # first 50 skin test samples
    emb = get_image_embedding(item["image"])
    sim_skin = torch.cosine_similarity(emb, avg_skin, dim=1).item()
    sim_radio = torch.cosine_similarity(emb, avg_radio, dim=1).item()
    pred = "skin" if sim_skin > sim_radio else "radiology"

    if pred == "skin":
        correct += 1
    total += 1

# Classify and evaluate radiology images
for item in radio_test.select(range(50)):  # first 50 radio test samples
    emb = get_image_embedding(item["image"])
    sim_skin = torch.cosine_similarity(emb, avg_skin, dim=1).item()
    sim_radio = torch.cosine_similarity(emb, avg_radio, dim=1).item()
    pred = "skin" if sim_skin > sim_radio else "radiology"

    if pred == "radiology":
        #print("radiology")
        correct += 1
    total += 1

# Accuracy
accuracy = correct / total * 100
print(f" Classification Accuracy (50 skin + 50 radiology): {accuracy:.2f}%")


 Classification Accuracy (50 skin + 50 radiology): 100.00%


###Loading Base, Skin and Radiology Models fro testing Testing

In [ ]:

#skin model
base_model, processor_base=BaseModel()

# Load LoRA Adapters
model_skin_test = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/skin_model/content/skin_model")

model_radio_test = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/radio_model/content/radio_model")
'''
# Load LoRA Adapters
model_skin_test = PeftModel.from_pretrained(base_model, "/content/skin_model")

model_radio_test = PeftModel.from_pretrained(base_model, "/content/radio_model")
'''


==((====))==  Unsloth 2025.7.11: Fast Gemma3N patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Gemma3N does not support SDPA - switching to eager!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/469M [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


'\n# Load LoRA Adapters\nmodel_skin_test = PeftModel.from_pretrained(base_model, "/content/skin_model")\n\nmodel_radio_test = PeftModel.from_pretrained(base_model, "/content/radio_model")\n'

###Testing Of Skin and Radiology Models
Testing after dynamic selection(as explained above)

In [ ]:


print("Skin test examples:", len(skin_test))
print("Radiology test examples:", len(radio_test))


FastVisionModel.for_inference(model_skin_test)  # Enable for inference!
FastVisionModel.for_inference(model_radio_test)  # Enable for inference!

def choose_model(image: Image.Image):

    emb = get_image_embedding(image)
    sim_skin = torch.cosine_similarity(emb, avg_skin, dim=1).item()
    sim_radio = torch.cosine_similarity(emb, avg_radio, dim=1).item()

    print(f"Similarity → Skin: {sim_skin:.3f}, Radiology: {sim_radio:.3f}")

    return "skin" if sim_skin > sim_radio else "radiology"



def run_pipeline(image: Image.Image):
    domain = choose_model(image)
    #question to ask based on dataset
    if domain == "skin":
      instruction = "You are an expert dermatalogist. Describe accurately what you see in this image."
    else:
      instruction = "You are an expert radiologist. Describe accurately what you see in this image."


     # Convert grayscale image to RGB
    if image.mode == 'L':
        image = image.convert('RGB')

    messages = [
       {
        "role": "user",
        "content": [{"type": "image"}, {"type": "text", "text": instruction}],
       }
        ]

    # Apply prompt template
    input_text = processor_base.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor_base(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")




    if domain == "skin":
        print("Using skin model")
        output_ids=model_skin_test.generate(**inputs, max_new_tokens = 256,
                        use_cache=True, temperature = 1.0, top_p = 0.95, top_k = 64)
        result = processor_base.decode(
        output_ids[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True
        )

    else:
        print("Using radiology model")
        output_ids=model_radio_test.generate(**inputs, max_new_tokens = 256,
                        use_cache=True, temperature = 1.0, top_p = 0.95, top_k = 64)
        result = processor_base.decode(
        output_ids[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True
        )
    return result



skin_results=[]
radio_results=[]




# skin model Testing
print("\n--- Running on Skin Test Set ---")
for idx, item in enumerate(skin_test.select(range(40))):
    print(f"\n[Skin] Sample {idx+1}/{len(skin_test)}")
    temp=run_pipeline(item["image"])

    skin_results.append({
        "prediction": temp,
        "ground_truth": item["answer"],
        })


# radio Model Testing
print("\n--- Running on Radiology Test Set ---")
for idx, item in enumerate(radio_test.select(range(40))):
    print(f"\n[Radiology] Sample {idx+1}/{len(radio_test)}")
    temp=run_pipeline(item["image"])
    radio_results.append({
        "prediction": temp,
        "ground_truth": item["answer"],
        })




Skin test examples: 50
Radiology test examples: 50

--- Running on Skin Test Set ---

[Skin] Sample 1/50
Similarity → Skin: 0.815, Radiology: 0.732
Using skin model

[Skin] Sample 2/50
Similarity → Skin: 0.808, Radiology: 0.689
Using skin model

[Skin] Sample 3/50
Similarity → Skin: 0.792, Radiology: 0.651
Using skin model

[Skin] Sample 4/50
Similarity → Skin: 0.675, Radiology: 0.619
Using skin model

[Skin] Sample 5/50
Similarity → Skin: 0.848, Radiology: 0.708
Using skin model

[Skin] Sample 6/50
Similarity → Skin: 0.832, Radiology: 0.688
Using skin model

[Skin] Sample 7/50
Similarity → Skin: 0.795, Radiology: 0.748
Using skin model

[Skin] Sample 8/50
Similarity → Skin: 0.802, Radiology: 0.701
Using skin model

[Skin] Sample 9/50
Similarity → Skin: 0.839, Radiology: 0.703
Using skin model

[Skin] Sample 10/50
Similarity → Skin: 0.830, Radiology: 0.710
Using skin model

[Skin] Sample 11/50
Similarity → Skin: 0.834, Radiology: 0.716
Using skin model

[Skin] Sample 12/50
Similarity →

##Testing Base Model

In [ ]:

FastVisionModel.for_inference(base_model)  # Enable for inference!
def base_model_run_pipeline(image: Image.Image):
      instruction = " Describe accurately what you see in this image."
      # Convert grayscale image to RGB
      if image.mode == 'L':
          image = image.convert('RGB')

      messages = [
       {
        "role": "user",
        "content": [{"type": "image"}, {"type": "text", "text": instruction}],
       }
        ]
      input_text = processor_base.apply_chat_template(messages, add_generation_prompt=True)
      inputs = processor_base(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
          ).to("cuda")


      output_ids=base_model.generate(**inputs, max_new_tokens = 256,
                        use_cache=True, temperature = 1.0, top_p = 0.95, top_k = 64)
      result = processor_base.decode(
      output_ids[0][inputs['input_ids'].shape[-1]:],skip_special_tokens=True)
      return result

base_results_skin=[]
base_results_radio=[]


for idx, item in enumerate(skin_test.select(range(40))):

    temp = base_model_run_pipeline(item["image"])
    base_results_skin.append({
        "prediction": temp,
        "ground_truth": item["answer"],
    })

for idx, item in enumerate(radio_test.select(range(40))):
    temp = base_model_run_pipeline(item["image"])
    base_results_radio.append({
        "prediction": temp,
        "ground_truth": item["answer"],
    })

##Finding accuracy of Models
Finding accuracy of models (Skin, Base, Radiology) using `all-mpnet-base-v2`, will pass the list of preticted outcome along ground truth to determine hte accuracy how models are predicting

In [ ]:
import torch
from PIL import Image
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

# 1. Load Sentence Embedding Model (lightweight but effective)
#embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embedder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


# 2. Function to compute cosine similarity between two texts
def get_similarity(pred: str, reference: str):
    emb1 = embedder.encode(pred, convert_to_tensor=True)
    emb2 = embedder.encode(reference, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(emb1, emb2)
    return similarity.item()

# 3. Evaluate dataset
def calculate_model_performance(result):
    similarities = []
    for data in result:
      #print(data["prediction"])
      similarity = get_similarity(data["prediction"], data["ground_truth"])
      similarities.append(similarity)

    average_similarity = sum(similarities) / len(similarities)
    return average_similarity


skin_similarity=0
radio_similarity=0
base_similarity_skin=0
base_similarity_radio=0
skin_similarity=calculate_model_performance(skin_results)
radio_similarity=calculate_model_performance(radio_results)
base_similarity_radio=calculate_model_performance(base_results_radio)
base_similarity_skin=calculate_model_performance(base_results_skin)


print("Base Model on Skin Set: ",base_similarity_skin)
print("Base Model on Radio Set: ",base_similarity_radio)

print("Skin Model Similarity: ",skin_similarity)
print("Radiology Model Similarity", radio_similarity)



Base Model on Skin Set:  0.49720655139535663
Base Model on Radio Set:  0.5082377038896084
Skin Model Similarity:  0.4984181011095643
Radiology Model Similarity 0.505672205798328




### Model Evaluation & Key Findings

For evaluating the performance of my image-based diagnosis model, I used an **embedding-based similarity method** with the help of:

```python
embedder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
```

This model compares the **text output** from my model (before and after training) with the **actual ground truth descriptions** by measuring sentence similarity.

#### Limitation of This Evaluation Method

While this approach gives a rough idea of how close the generated description is to the actual answer, it’s **not always reliable** for measuring accuracy in medical or clinical settings. This is because:

* It focuses more on **how similar the words and sentence structure are**, not on **clinical correctness** or **meaningful diagnosis**.
* I tested it by asking ChatGPT to compare both outputs (before and after training), and it clearly preferred the trained model.
* However, the similarity score from SentenceTransformer showed **only a very small improvement**, which i believe doesn’t reflect the **true progress** in medical accuracy.

#### Model Training Notes

* Both versions of the model (before and after training) were run for only **2 epochs** due to limited GPU resources on Google Colab.
* Even in just 2 epochs, the **training loss dropped from 10.0 to 0.7**, which is a **massive improvement** — showing that the model learned significantly even with limited training.
* The training process took **over 1 hour**, and I had to switch between multiple Google accounts to manage Colab limits.
* With **more training epochs**, **a larger dataset**, and **better computing power**, the model’s accuracy and output quality would improve even further.



### Conclusion

Even with minimal training, the model shows significant learning progress (based on loss reduction and qualitative evaluation). However, the sentence similarity metric alone is **not enough to judge model accuracy**, especially in medical contexts. A better approach would involve human validation or domain-specific evaluation metrics.




##Saving Results
Saving model results (predicted & Actual) to compute the accuracy using another way

In [ ]:
import os

# Create the base folder
folder_name = 'model_outputs'
os.makedirs(folder_name, exist_ok=True)



# Store them in a dictionary for easy saving
all_results = {
    'skin_model': skin_results,
    'radio_model': radio_results,
    'base_skin_model': base_results_skin,
    'base_radio_model': base_results_radio,
}

# Save each to its own file
for model_name, result_list in all_results.items():
    file_path = os.path.join(folder_name, f'{model_name}.txt')
    with open(file_path, 'w') as f:
        for result in result_list:
            f.write(f"{result}\n")


In [ ]:
import shutil

# Name of the folder to zip
folder_name = 'model_outputs'

# Name of the zip file to create
zip_filename = f'{folder_name}.zip'

# Create the zip file
shutil.make_archive(folder_name, 'zip', folder_name)


'/content/model_outputs.zip'